In [1]:
# CÁLCULO DA ESTATÍSTICA DESCRITIVA (ABORDAGEM IID)

import io
import math
import numpy as np
import pandas as pd

# Upload do CSV
try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
except Exception as e:
    raise RuntimeError("Falha ao carregar o arquivo. Tente novamente o upload do CSV.") from e

# Parse de datas
if 'Data' not in df.columns or 'Consumo' not in df.columns:
    raise ValueError("O CSV precisa conter as colunas 'Data' e 'Consumo'.")

df['Data'] = pd.to_datetime(df['Data'])

# Filtra março e horário 04:00
mask_marco_4h = (df['Data'].dt.month == 3) & (df['Data'].dt.hour == 4)
amostra = df.loc[mask_marco_4h, 'Consumo'].astype(float).reset_index(drop=True)

if amostra.empty:
    raise ValueError("Não foram encontrados registros para março às 04:00 no arquivo fornecido.")

# --- Estatísticas ---
n = amostra.shape[0]
media = amostra.mean()
mediana = amostra.median()

# Modas (podem existir múltiplas)
modas = amostra.mode().tolist()
moda = modas[0] if len(modas) > 0 else np.nan  # moda principal (primeira)

minimo = amostra.min()
maximo = amostra.max()
amplitude = maximo - minimo

# Desvio-padrão e variância amostrais (ddof=1)
desvio_padrao_amostral = amostra.std(ddof=1)
variancia_amostral = amostra.var(ddof=1)

# Coeficiente de Variação (CV = s / média). Se média=0, define como NaN
coef_var = desvio_padrao_amostral / media if media != 0 else np.nan

# Coeficiente de Assimetria (skewness) com correção de viés.
# Preferimos scipy (se disponível). Caso contrário, usamos pandas (que já aplica correção).
def skewness_series(s: pd.Series) -> float:
    try:
        from scipy.stats import skew  # type: ignore
        return float(skew(s.values, bias=False, nan_policy='omit'))
    except Exception:
        return float(s.skew())  # pandas: já é a versão amostral

coef_assimetria = skewness_series(amostra)

# --- Impressão formatada ---
print("Estatísticas — Consumo às 04h em Março")
print("-" * 45)
print(f"Tamanho da amostra:             {n}")
print(f"Média:                          {media:.4f}")
print(f"Mediana:                        {mediana:.4f}")
if len(modas) == 1:
    print(f"Moda:                           {moda:.4f}")
else:
    # Exibe todas as modas, se houver mais de uma
    modas_fmt = ", ".join(f"{m:.4f}" for m in modas)
    print(f"Modas ({len(modas)}):                 {modas_fmt}")
print(f"Mínimo:                         {minimo:.4f}")
print(f"Máximo:                         {maximo:.4f}")
print(f"Amplitude:                      {amplitude:.4f}")
print(f"Desvio-padrão (amostral):       {desvio_padrao_amostral:.4f}")
print(f"Variância da amostra:           {variancia_amostral:.4f}")
print(f"Coeficiente de Variação:        {coef_var:.6f}")
print(f"Coeficiente de Assimetria:      {coef_assimetria:.6f}")

# (Opcional) Mostrar um resumo rápido por ano: média de março às 4h por ano
resumo_ano = (
    df.loc[mask_marco_4h, ['Data', 'Consumo']]
      .assign(Ano=lambda x: x['Data'].dt.year)
      .groupby('Ano')['Consumo'].agg(['count', 'mean', 'min', 'max'])
      .rename(columns={'count': 'N', 'mean': 'Media', 'min': 'Min', 'max': 'Max'})
)
print("\nResumo por ano (março às 04:00):")
print(resumo_ano)


Saving TRABALHO ESTOCÁSTICOS4 - Final - Base.csv to TRABALHO ESTOCÁSTICOS4 - Final - Base.csv
Estatísticas — Consumo às 04h em Março
---------------------------------------------
Tamanho da amostra:             403
Média:                          13758.0223
Mediana:                        13737.0000
Modas (2):                 12884.0000, 13510.0000
Mínimo:                         9917.0000
Máximo:                         19240.0000
Amplitude:                      9323.0000
Desvio-padrão (amostral):       1700.2358
Variância da amostra:           2890801.9174
Coeficiente de Variação:        0.123581
Coeficiente de Assimetria:      0.196110

Resumo por ano (março às 04:00):
       N         Media    Min    Max
Ano                                 
2005  31  14239.774194  11552  16830
2006  31  14196.709677  11420  16213
2007  31  13983.612903  11272  16873
2008  31  14980.322581  13567  17180
2009  31  13422.258065  10891  18956
2010  31  13610.193548  11477  16453
2011  31  14105.774194 

In [ ]:
# CÁLCULO PARA O BOX PLOT MODIFICADO (ABORDAGEM IID)

import io
import pandas as pd

# Upload do CSV
try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
except Exception as e:
    raise RuntimeError("Falha ao carregar o arquivo. Refaça o upload do CSV.") from e

# Checagens básicas
if 'Data' not in df.columns or 'Consumo' not in df.columns:
    raise ValueError("O CSV precisa conter as colunas 'Data' e 'Consumo'.")

# Preparação
df['Data'] = pd.to_datetime(df['Data'])

# Filtra março e horário 04:00
amostra_df = df[(df['Data'].dt.month == 3) & (df['Data'].dt.hour == 4)][['Data','Consumo']].copy()
amostra = amostra_df['Consumo'].astype(float)

if amostra.empty:
    raise ValueError("Não há registros para março às 04:00 no arquivo fornecido.")

# Quartis e limites do box plot modificado
Q1 = amostra.quantile(0.25)
Q3 = amostra.quantile(0.75)
AIQ = Q3 - Q1
Li = Q1 - 1.5 * AIQ
Ls = Q3 + 1.5 * AIQ

# Whiskers (mínimo/máximo dentro dos limites)
whisker_min = amostra[amostra >= Li].min()
whisker_max = amostra[amostra <= Ls].max()

# Outliers moderados
outliers_baixos = amostra_df[amostra_df['Consumo'] < Li].copy()
outliers_altos  = amostra_df[amostra_df['Consumo'] > Ls].copy()

outliers_baixos['Tipo'] = 'Abaixo de Li'
outliers_altos['Tipo']  = 'Acima de Ls'

outliers = pd.concat([outliers_baixos, outliers_altos], ignore_index=True).sort_values('Data').reset_index(drop=True)

# Impressão dos resultados
print("Parâmetros do Box Plot Modificado — Consumo às 04h em Março")
print("-" * 70)
print(f"Tamanho da amostra (n):  {len(amostra)}")
print(f"Q1 (1º quartil):         {Q1:.4f}")
print(f"Q3 (3º quartil):         {Q3:.4f}")
print(f"AIQ (Q3 - Q1):           {AIQ:.4f}")
print(f"Limite inferior (Li):    {Li:.4f}")
print(f"Limite superior (Ls):    {Ls:.4f}")
print(f"Whisker mínimo:          {whisker_min:.4f}")
print(f"Whisker máximo:          {whisker_max:.4f}")

print("\nResumo de outliers moderados:")
print(f" - Abaixo de Li: {len(outliers_baixos)}")
print(f" - Acima de Ls:  {len(outliers_altos)}")
print(f" - Total:        {len(outliers)}")

# Mostra alguns outliers, se existirem
if not outliers.empty:
    display_cols = ['Data', 'Consumo', 'Tipo']
    try:
        from IPython.display import display
        display(outliers[display_cols])
    except Exception:
        print("\nOutliers (primeiras linhas):")
        print(outliers[display_cols].head())

# (Opcional) salvar CSV com os outliers
salvar_csv = True
if salvar_csv and not outliers.empty:
    nome_arquivo = "outliers_marco_4h.csv"
    outliers.to_csv(nome_arquivo, index=False)
    try:
        files.download(nome_arquivo)
    except Exception:
        print(f"\nArquivo salvo em disco: {nome_arquivo}")


In [ ]:
# HISTOGRAMA + TESTE KS +  TESTE DE LJUNG–BOX (ABORDAGEM IID)

import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Imports estatísticos
from scipy import stats
from statsmodels.stats.diagnostic import acorr_ljungbox

# === Upload do CSV (Colab) ===
try:
    from google.colab import files  # type: ignore
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
except Exception as e:
    raise RuntimeError("Falha ao carregar o arquivo. Faça o upload do CSV e tente novamente.") from e

# === Checagens básicas ===
if 'Data' not in df.columns or 'Consumo' not in df.columns:
    raise ValueError("O CSV precisa conter as colunas 'Data' e 'Consumo'.")

# === Preparação da amostra: março às 04:00 ===
df['Data'] = pd.to_datetime(df['Data'])
amostra_df = df[(df['Data'].dt.month == 3) & (df['Data'].dt.hour == 4)][['Data','Consumo']].copy()

if amostra_df.empty:
    raise ValueError("Não há registros para março às 04:00 no arquivo fornecido.")

amostra = amostra_df['Consumo'].astype(float).dropna().reset_index(drop=True)

# === Parâmetros da amostra ===
n = len(amostra)
media = amostra.mean()
# Desvio-padrão amostral (ddof=1)
desvio = amostra.std(ddof=1)

# === Número de classes pelo método de Rice ===
# k = ceil(2 * n^(1/3))
k_rice = int(np.ceil(2 * n ** (1/3)))

# === Plot: histograma (densidade) + curva normal ajustada ===
plt.figure(figsize=(8, 5))
# Histograma como densidade para compatibilizar com a PDF normal
contagens, bin_edges, _ = plt.hist(amostra, bins=k_rice, density=True, alpha=0.5)
# Intervalo para a curva
x = np.linspace(amostra.min(), amostra.max(), 400)
pdf_normal = stats.norm.pdf(x, loc=media, scale=desvio)
plt.plot(x, pdf_normal, linewidth=2)

plt.title("Consumo às 04:00 em Março — Histograma (Rice) + Normal Ajustada")
plt.xlabel("Consumo")
plt.ylabel("Densidade")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.show()

# === Teste KS para N(media, desvio^2) ===
# Observação: parâmetros estimados da amostra => p-valor do KS é aproximado (Lilliefors seria o ideal).
ks_stat, ks_p = stats.kstest(amostra, 'norm', args=(media, desvio))

print("=== Teste Kolmogorov–Smirnov para Normal Ajustada ===")
print(f"n = {n}")
print(f"Média = {media:.6f} | Desvio-padrão(amostral) = {desvio:.6f}")
print(f"Bins (Rice) = {k_rice}")
print(f"KS estatística = {ks_stat:.6f} | p-valor = {ks_p:.6f}")

alpha = 0.05
if ks_p < alpha:
    print(f"Decisão (α={alpha:.2f}): Rejeitar H0 (a amostra difere da Normal ajustada).")
else:
    print(f"Decisão (α={alpha:.2f}): Não rejeitar H0 (compatível com Normal ajustada).")

# === Teste Ljung–Box (lag = 10) ===
# H0: ausência de autocorrelação até a defasagem indicada.
lb_df = acorr_ljungbox(amostra, lags=[10], return_df=True)
lb_stat = float(lb_df['lb_stat'].iloc[0])
lb_p = float(lb_df['lb_pvalue'].iloc[0])

print("\n=== Teste Ljung–Box ===")
print(f"Lag = 10 | Estatística = {lb_stat:.6f} | p-valor = {lb_p:.6e}")
if lb_p < alpha:
    print(f"Decisão (α={alpha:.2f}): Rejeitar H0 — evidência de autocorrelação (não-aleatoriedade).")
else:
    print(f"Decisão (α={alpha:.2f}): Não rejeitar H0 — sem evidência de autocorrelação.")


In [ ]:
# TESTE DE ESTACIONARIEDADE ESTACIONARIEDADE (ABORDAGEM DE SÉRIES TEMPORAIS)

import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

# === 1) Ler o CSV ===
# Ajuste o caminho se necessário
csv_path = "TRABALHO ESTOCÁSTICOS4 - Final - Base.csv"
df = pd.read_csv(csv_path)

# === 2) Detectar e converter a coluna de data/hora (sem infer_datetime_format) ===
# tenta nomes comuns; se não achar, tenta a 1ª coluna de texto parseável
candidatas_dt = [c for c in df.columns
                 if c.lower() in ["datetime","data","date","timestamp","time","hora","dt"]]

if candidatas_dt:
    dt_col = candidatas_dt[0]
else:
    dt_col = None
    for c in df.select_dtypes(include="object").columns:
        try:
            pd.to_datetime(df[c], errors="raise")
            dt_col = c
            break
        except Exception:
            pass
    if dt_col is None:
        raise ValueError("Não foi possível localizar uma coluna de data/hora; informe-a em dt_col.")

df["dt"] = pd.to_datetime(df[dt_col], errors="coerce")  # <-- sem infer_datetime_format
df = df.dropna(subset=["dt"]).sort_values("dt").set_index("dt")

# === 3) Detectar a coluna numérica de consumo ===
num_cols = df.select_dtypes(include="number").columns.tolist()
if not num_cols:
    # tenta converter possíveis colunas com vírgula decimal
    for c in df.columns:
        if c != "dt":
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")
    num_cols = df.select_dtypes(include="number").columns.tolist()

if not num_cols:
    raise ValueError("Não foi encontrada coluna numérica de consumo.")
valor_col = num_cols[0] if len(num_cols) == 1 else df[num_cols].count().idxmax()

# === 4) Recorte: todo novembro de 2016 ===
ts_nov2016 = df.loc["2016-11-01":"2016-11-30 23:59:59", valor_col].dropna()

# Sanidade: precisa de > 0 observações
if ts_nov2016.empty:
    raise ValueError("O recorte de novembro/2016 ficou vazio. Confira o fuso/colunas/arquivo.")

# === 5) Teste ADF ===
adf_stat, p_value, usedlag, nobs, crit_vals, icbest = adfuller(ts_nov2016.values, autolag="AIC")

print("Período analisado: 2016-11-01 a 2016-11-30")
print(f"Tamanho da amostra: {int(nobs)}")
print(f"Estatística ADF: {adf_stat:.3f}")
print(f"Valor-p: {p_value:.3g}")
print("Valores críticos:")
for k, v in crit_vals.items():
    print(f"  {k}%: {v:.3f}")

alpha = 0.05
conclusao = ("Série estacionária (rejeita H0 de raiz unitária)"
             if p_value < alpha else
             "Série NÃO estacionária (não rejeita H0 de raiz unitária)")
print("Conclusão (α=5%):", conclusao)


In [ ]:
# SÉRIE DIFERENCIADA + GRÁFICO ACF + GRÁFICO PACF (ABORDAGEM DE SÉRIES TEMPORAIS)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ---------- Config ----------
csv_path = "TRABALHO ESTOCÁSTICOS4 - Final - Base.csv"  # ajuste se estiver em outro caminho
nome_col_data_preferidos = ["Data","datetime","date","timestamp","hora","time","dt"]
nome_col_valor_preferidos = ["Consumo","valor","load","mw","aep_mw"]

# ---------- Leitura ----------
df = pd.read_csv(csv_path)

# Detecta coluna de data
dt_col = None
for c in df.columns:
    if c.lower() in [n.lower() for n in nome_col_data_preferidos]:
        dt_col = c
        break
if dt_col is None:
    for c in df.select_dtypes(include="object").columns:
        try:
            pd.to_datetime(df[c], errors="raise")
            dt_col = c
            break
        except Exception:
            pass
if dt_col is None:
    raise ValueError("Não encontrei coluna de data/hora. Informe-a manualmente (ex.: dt_col = 'Data').")

# Converte datas (sem infer_datetime_format) e ordena
df["dt"] = pd.to_datetime(df[dt_col], errors="coerce")
df = df.dropna(subset=["dt"]).sort_values("dt").set_index("dt")

# Detecta coluna numérica de consumo
val_col = None
for c in df.columns:
    if c == "dt":
        continue
    if c.lower() in [n.lower() for n in nome_col_valor_preferidos]:
        val_col = c
        break
if val_col is None:
    # tenta numéricas diretas
    num_cols = df.select_dtypes(include="number").columns.tolist()
    if not num_cols:
        # tenta converter trocando vírgula por ponto
        for c in df.columns:
            if c != "dt":
                df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")
        num_cols = df.select_dtypes(include="number").columns.tolist()
    if not num_cols:
        raise ValueError("Não encontrei coluna numérica de consumo. Defina manualmente (ex.: val_col = 'Consumo').")
    # escolhe a com mais dados
    val_col = df[num_cols].count().idxmax()

# ---------- Recorte novembro/2016 ----------
ts = df[val_col].astype(float).sort_index()
ts_nov = ts.loc["2016-11-01":"2016-11-30 23:59:59"].dropna()
if ts_nov.empty:
    raise ValueError("Sem dados para novembro/2016 no arquivo.")

# ---------- Diferenciação d = 1 ----------
ts_diff = ts_nov.diff().dropna()

# ---------- Plot: série diferenciada ----------
plt.figure(figsize=(10,4))
plt.plot(ts_diff.index, ts_diff.values)
plt.title("Série Diferenciada (d = 1) — Novembro/2016")
plt.xlabel("Data")
plt.ylabel("Δ Consumo")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()

# ---------- Plot: ACF ----------
plt.figure(figsize=(8,4))
plot_acf(ts_diff, lags=40, alpha=0.05)  # 40 defasagens é usual para base horária mensal
plt.title("ACF da Série Diferenciada (d = 1)")
plt.tight_layout()
plt.show()

# ---------- Plot: PACF ----------
plt.figure(figsize=(8,4))
plot_pacf(ts_diff, lags=40, alpha=0.05, method="ywm")  # Yule-Walker modificado é estável
plt.title("PACF da Série Diferenciada (d = 1)")
plt.tight_layout()
plt.show()

print(f"N observações em nov/2016: {len(ts_nov)} | Após diff: {len(ts_diff)}")


In [ ]:
# MODELO MATEMÁTICO DO ARIMA(1,1,1)- (ABORDAGEM DE SÉRIES TEMPORAIS)

!pip -q install statsmodels pandas

import pandas as pd
import statsmodels.api as sm

# --- 1) Upload do CSV ---
from google.colab import files
uploaded = files.upload()   # selecione: TRABALHO ESTOCÁSTICOS4 - Final - Base.csv
csv_path = list(uploaded.keys())[0]

# --- 2) Ler dados ---
df = pd.read_csv(csv_path)
df["Data"] = pd.to_datetime(df["Data"])

# --- 3) Filtrar novembro/2016 ---
serie = df.loc[(df["Data"].dt.year==2016) & (df["Data"].dt.month==11), "Consumo"]

# --- 4) Ajustar ARIMA(1,1,1) ---
modelo = sm.tsa.ARIMA(serie, order=(1,1,1))
resultado = modelo.fit()

# --- 5) Exibir coeficientes ---
print("Resumo:")
print(resultado.summary())

print("\nCoeficientes principais:")
print("AR(1) =", resultado.params["ar.L1"])
print("MA(1) =", resultado.params["ma.L1"])
print("σ²    =", resultado.params["sigma2"])


In [ ]:
# CÁLCULO DO MAPE- (ABORDAGEM DE SÉRIES TEMPORAIS)

!pip -q install statsmodels pandas numpy

import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# 1) Upload do arquivo CSV
from google.colab import files
uploaded = files.upload()  # selecione: TRABALHO ESTOCÁSTICOS4 - Final - Base.csv
csv_path = list(uploaded.keys())[0]
print("Arquivo carregado:", csv_path)

# 2) Ler base e filtrar nov/2016
df = pd.read_csv(csv_path)
df["Data"] = pd.to_datetime(df["Data"])
mask = (df["Data"].dt.year == 2016) & (df["Data"].dt.month == 11)
y = df.loc[mask, "Consumo"].astype(float).reset_index(drop=True)

print("Tamanho da amostra (nov/2016):", len(y))  # esperado: 721

# 3) Ajustar ARIMA(1,1,1)
model = ARIMA(y, order=(1, 1, 1))
res = model.fit()

# 4) Previsão one-step ahead in-sample (dynamic=False) e MAPE
fitted = res.fittedvalues              # já no espaço de Y_t
y_aligned = y.copy()
fitted_aligned = fitted.reindex_like(y_aligned)

mask_valid = ~fitted_aligned.isna() & (y_aligned != 0)
mape = (np.abs((y_aligned[mask_valid] - fitted_aligned[mask_valid]) / y_aligned[mask_valid])).mean() * 100

# 5) Coeficientes e equação
phi1 = res.params.get("ar.L1", np.nan)
theta1 = res.params.get("ma.L1", np.nan)
sigma2 = res.params.get("sigma2", np.nan)

print("\n=== Coeficientes ARIMA(1,1,1) ===")
print(f"AR(1)  = {phi1:.6f}")
print(f"MA(1)  = {theta1:.6f}")
print(f"sigma^2= {sigma2:.6f}")

print("\n=== Equação (d=1) ===")
print("ΔY_t = φ1 * ΔY_{t-1} + ε_t + θ1 * ε_{t-1}")
print(f"ΔY_t = {phi1:.4f} * ΔY_(t-1) + ε_t + {theta1:.4f} * ε_(t-1)")

print("\n=== MAPE in-sample (one-step ahead) ===")
print(f"MAPE = {mape:.3f}%")

# (Opcional) mostrar 10 primeiras previsões vs observados
preview = pd.DataFrame({
    "Y_observado": y_aligned,
    "Y_previsto_1passo": fitted_aligned
}).dropna().head(10)
print("\nObservado vs. Previsto (primeiros 10 pontos):")
print(preview.to_string(index=False))


In [ ]:
# TESTE  LJUNG-BOX NOS RESÍDUOS- (ABORDAGEM DE SÉRIES TEMPORAIS)
!pip -q install statsmodels pandas numpy

import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

# 1) Upload do CSV
from google.colab import files
uploaded = files.upload()  # escolha: TRABALHO ESTOCÁSTICOS4 - Final - Base.csv
csv_path = list(uploaded.keys())[0]

# 2) Leitura e filtro nov/2016
df = pd.read_csv(csv_path)
df["Data"] = pd.to_datetime(df["Data"])
mask = (df["Data"].dt.year == 2016) & (df["Data"].dt.month == 11)
y = df.loc[mask, "Consumo"].astype(float).reset_index(drop=True)

# 3) Ajuste ARIMA(1,1,1)
res = ARIMA(y, order=(1,1,1)).fit()

# 4) Resíduos in-sample (one-step, dynamic=False) e Ljung-Box
resid = res.resid.dropna()

# lags que você quer avaliar
lags_to_test = [10, 20, 30]
lb = acorr_ljungbox(resid, lags=lags_to_test, return_df=True)

print("=== Coeficientes do ARIMA(1,1,1) ===")
print(f"AR(1) = {res.params.get('ar.L1'): .6f}")
print(f"MA(1) = {res.params.get('ma.L1'): .6f}\n")

print("=== Teste de Ljung-Box nos resíduos ===")
# renomear colunas para clareza
lb = lb.rename(columns={"lb_stat": "Estatística", "lb_pvalue": "p_valor"})
print(lb.to_string(index=True))

# 5) Interpretação simples (alpha=0.05)
alpha = 0.05
for lag, p in zip(lb.index, lb["p_valor"]):
    concl = "NÃO rejeita autocorrelação (OK)" if p > alpha else "Rejeita: há autocorrelação"
    print(f"Lag {lag:>2}: p={p:.3g} → {concl}")
